### Import necessary modules

In [96]:
import os
import zipfile
import pandas as pd
from IPython.display import display
from IPython.display import Javascript
import requests

### Important constants (expected from user)

In [97]:
DATA_DIR = "firefox_data"

### Unpack archives and get tables

In [98]:
FILES_DIR = f"{DATA_DIR}_files"

In [99]:
DFS = dict()
for archive in os.listdir(DATA_DIR):
    path = os.path.join(DATA_DIR, archive)
    with zipfile.ZipFile(path, 'r') as zip_ref:
        zip_ref.extractall(FILES_DIR)
    DFS[archive.split('_')[0]] = pd.read_csv(f"{os.path.join(FILES_DIR, archive)[:-3]}csv")

### Define utility functions

Comments:
* All functions are able to work with the beginning of a key
* Key is usially OS

In [100]:
def head(
    key: str,
    number_of_rows: int=5,
    show: bool=False
):
    for df_key in DFS:
        if df_key.startswith(key):
            df_head = DFS[df_key].head(number_of_rows)
            if show:
                display(df_head)
            return df_head

In [101]:
# If keys == None, all keys will be used
def find_instruction(
    instruction: str,
    keys: None|list[str]=None,
    only_filenames: bool=True,
    show: bool=False,
    number_of_rows_to_show: int=5
):
    if keys is None:
        keys = list(DFS)
    tables = dict()
    for key in keys:
        for df_key in DFS:
            if df_key.startswith(key):
                try:
                    table = DFS[df_key][DFS[df_key][instruction] != 0]
                    if only_filenames:
                        table = list(table.filename)
                    tables[df_key] = table
                except KeyError:
                    tables[df_key] = None
    if show:
        for key in tables:
            print(f"{key}:")
            if tables[key] is None:
                continue
            if only_filenames:
                for filename in tables[key]:
                    print(f"\t{filename}")
            else:
                display(tables[key].head(number_of_rows_to_show))
            print()
            
    return tables

In [112]:
def what_is_instruction(
    instruction: str
):
    page_text = requests.get('https://www.felixcloutier.com/x86/').text
    tmp = page_text[:(page_text.find(f'>{instruction.upper()}<') - 1)]
    path_part = tmp[-(tmp[::-1].index('"') - 1):]
    display(Javascript('window.open("{url}");'.format(url=f"https://www.felixcloutier.com/x86/{path_part}")))

### Experiments

In [8]:
head('m', 10)

,filename,endbr64,sub,test,call,add,ret,push,pxor,mov,...,vfnmsub132pd,vmovntdqa,vfnmsub132ps,vmovntpd,vcvtpd2dqy,pmaxsb,pminsb,movntpd,aad,into
0,/usr/bin/nf-exp-add,6,6,5,1,5,7,9,1,100,...,0,0,0,0,0,0,0,0,0,0
1,/usr/bin/asn1Coding,6,9,30,1,16,7,42,0,282,...,0,0,0,0,0,0,0,0,0,0
2,/usr/bin/lslocks,9,21,92,1,30,14,36,0,497,...,0,0,0,0,0,0,0,0,0,0
3,/usr/bin/csplit,9,94,204,1,126,44,129,12,1426,...,0,0,0,0,0,0,0,0,0,0
4,/usr/bin/cksum,34,104,225,9,1096,49,189,29,3045,...,0,0,0,0,0,0,0,0,0,0
5,/usr/bin/chgpasswd,27,73,398,687,130,65,162,4,1820,...,0,0,0,0,0,0,0,0,0,0
6,/usr/bin/nl,8,45,132,1,88,26,79,5,985,...,0,0,0,0,0,0,0,0,0,0
7,/usr/bin/dbus-uuidgen,6,5,15,1,9,8,14,0,59,...,0,0,0,0,0,0,0,0,0,0
8,/usr/bin/db_verify,7,8,37,30,7,9,15,0,198,...,0,0,0,0,0,0,0,0,0,0
9,/usr/bin/ptx,11,171,345,1,250,49,135,9,2166,...,0,0,0,0,0,0,0,0,0,0


In [37]:
find_instruction('aad', show=True);

manjaro:
	/DataCollection/venv/lib/python3.10/site-packages/pip/_vendor/distlib/t32.exe

opensuse:
	/usr/lib/python3.10/site-packages/pip/_vendor/distlib/t32.exe
	/DataCollection/venv/lib/python3.10/site-packages/pip/_vendor/distlib/t32.exe

ubuntu:


In [113]:
what_is_instruction('nop')

<IPython.core.display.Javascript object>